In [4]:
from operator import itemgetter
from utils import merge_utils as mu
from utils import write_utils as wu

In [5]:
def voting(sc, rdds):
    ensemble_rdd = sc.emptyRDD()
    
    for rdd in rdds:
        ensemble_rdd = ensemble_rdd.union(rdd.map(lambda x: (x[0], [(i,1) for i in x[1]])))
    
    voting_rdd = (ensemble_rdd.flatMap(lambda r: [(r[0], x) for x in r[1]])
                     .map(lambda x: ((x[0],x[1][0]),x[1][1]))
                     .reduceByKey(lambda x,y:x+y)
                     .map(lambda x: (x[0][0],[(x[0][1],x[1])]))
                     .reduceByKey(lambda x,y:x+y)
                     .map(lambda x: (x[0],sorted(x[1], key=itemgetter(1), reverse=True)[:30]))
                     )
    
    return voting_rdd

def voting_weighted(sc, rdds):
    ensemble_rdd = sc.emptyRDD()
    
    for rdd in rdds:
        value = rdd[1]
        ensemble_rdd = ensemble_rdd.union(rdd[0].map(lambda x: (x[0], [(i,value) for i in x[1]])))
    
    voting_rdd = (ensemble_rdd.flatMap(lambda r: [(r[0], x) for x in r[1]])
                     .map(lambda x: ((x[0],x[1][0]),x[1][1]))
                     .reduceByKey(lambda x,y:x+y)
                     .map(lambda x: (x[0][0],[(x[0][1],x[1])]))
                     .reduceByKey(lambda x,y:x+y)
                     .map(lambda x: (x[0],sorted(x[1], key=itemgetter(1), reverse=True)[:30]))
                     )
    
    return voting_rdd

In [6]:
def recommendation_list(string):
    lista=[]
    split = string.split(',')
    for i in split:
        if i != '':
            lista.append((int(i)))
            
    return lista

def parse_submission(sc, path):
    rdd = sc.textFile(path)
    
    header = rdd.first()

    rdd = (rdd.filter(lambda x: x != header)
    .map(lambda x: x.split("\t"))
    .filter(lambda x: x[0] != '')
    .map(lambda x:(int(x[0]),recommendation_list(x[1])))
    )

    return rdd

In [7]:
interaction_rdd = parse_submission(sc, "../../Submissions/ens_subs/interactions_82k.csv")
impression_rdd = parse_submission(sc, "../../Submissions/ens_subs/impressions_116k.csv")

cf_intint_rdd = parse_submission(sc, "../../Submissions/ens_subs/cf_int_int_18.9k.csv")
cf_intimp_rdd = parse_submission(sc, "../../Submissions/ens_subs/cf_int_imp_22.4k.csv")
cf_impint_rdd = parse_submission(sc, "../../Submissions/ens_subs/imp_imp_int_int_33.2k_avg.csv")
cf_impimp_rdd = parse_submission(sc, "../../Submissions/ens_subs/cf_imp_imp_31k.csv")
cf_i_intint_rdd = parse_submission(sc, "../../Submissions/ens_subs/true_10k_CF_ITEM_greater_3_shr=1_knn=750_22k.csv")
cf_i_impimp_rdd = parse_submission(sc, "../../Submissions/ens_subs/cf_item_imp_imp_20k.csv")

cb_idf_rdd = parse_submission(sc, "../../Submissions/ens_subs/idf_45k.csv")
cb_concept_rdd = parse_submission(sc, "../../Submissions/ens_subs/tf_ervin_37k.csv")

In [8]:
recommendation_rdd = voting(sc, [interaction_rdd,
                                impression_rdd,
                                cf_intint_rdd,
                                cf_intimp_rdd,
                                cf_impimp_rdd,
                                cf_impint_rdd,
                                cf_i_impimp_rdd,
                                cf_i_intint_rdd,
                                cb_idf_rdd,
                                cb_concept_rdd])

In [9]:
wu.prepareSubmission(recommendation_rdd,"results/pure_voting")

Submission file results/pure_voting created successfully


In [12]:
recommendation_rdd = voting_weighted(sc, [(interaction_rdd,4),
                                (impression_rdd,4),
                                (cf_intint_rdd,1),
                                (cf_intimp_rdd,1),
                                (cf_impimp_rdd,1),
                                (cf_impint_rdd,1),
                                (cf_i_impimp_rdd,1),
                                (cf_i_intint_rdd,1),
                                (cb_idf_rdd,1),
                                (cb_concept_rdd,1)])

In [13]:
wu.prepareSubmission(recommendation_rdd,"results/weighted_voting")

Submission file results/weighted_voting created successfully
